<a href="https://colab.research.google.com/github/LuAsm/simple_python_project/blob/master/Copy_of_titanic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.6/518.6 KB 12.4 MB/s eta 0:00:00


In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.2 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=5a9d4b38a7b2e9e424dc6d6eee5c4bd40f6681446971c93016015e37aecbe8e2
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools


In [ ]:
import torch
import pytorch_lightning as pl
import pandas as pd
import requests
import tempfile
from typing import List
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer
from pytorch_lightning.loggers import WandbLogger
from time import time
import wandb

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Task 1
Write a `Dataset` for Titanic Survivors data. Use columns `sex`, `age`, `sibsp` and `parch`. Apply data transformations where needed. Fill the missing values if there are any. 

Split the dataset into train, val, and test parts.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
data["Age"].fillna(data["Age"].mean(), inplace=True)

In [ ]:
data["Sex"] = (data["Sex"] == "male").astype(int)

In [ ]:
data['SibSp'].unique()

array([1, 0, 3, 4, 2, 5, 8])

In [ ]:
data['Parch'].unique()

array([0, 1, 2, 5, 3, 4, 6])

In [ ]:
x = data[["Sex","Age","SibSp","Parch"]]

In [ ]:
x.head()

,Sex,Age,SibSp,Parch
0,1,22.0,1,0
1,0,38.0,1,0
2,0,26.0,0,0
3,0,35.0,1,0
4,1,35.0,0,0


In [ ]:
class TitanicDataset(Dataset):
  def __init__(self):
    self.x = data[["Sex","Age","SibSp","Parch"]]
    self.y = data[["Survived"]]
    self._scaler = StandardScaler()
    self._scaler.fit(self.x)

  def transformations(self):
    self.x["Sex"] = (self.x["Sex"] == "male").astype(int)

  def fill_missing_values(self):
    self.x["Age"].fillna(self.x["Age"].mean(), inplace=True)
    

In [ ]:
class TitanicDataset(Dataset):
  def __init__(self, df: pd.DataFrame, feature_names: List[str]):
    self._feature_names = feature_names
    self._scaler = StandardScaler()
    self._imputer = SimpleImputer(strategy="mean")

    self.data = self._process_data(df)
    
  def _process_data(self, df):
    df = df[["Survived", "Sex", "Age", "SibSp", "Parch"]]
    df["Sex"] = df["Sex"].apply(lambda x: int(x == "female"))

    self._imputer.fit(df[self._feature_names].values)
    self._scaler.fit(df[self._feature_names].values)

    return df

  def __len__(self):
    return self.data.shape[0]
    
  def __getitem__(self, idx):
    x = self.data.iloc[idx][self._feature_names].values
    y = self.data.iloc[idx]["Survived"]
    x_imputed = self._imputer.transform(x.reshape(1, -1))
    x_scaled = self._scaler.transform(x_imputed)[0]
    return x_scaled, y



In [ ]:
train_df, test_df = train_test_split(data, train_size=0.2, random_state=42)
test_df, val_df = train_test_split(test_df, train_size=0.5, random_state=42)

train_dataset = TitanicDataset(train_df, ["Age", "SibSp", "Parch", "Sex"])
test_dataset = TitanicDataset(test_df, ["Age", "SibSp", "Parch", "Sex"])
val_dataset = TitanicDataset(val_df, ["Age", "SibSp", "Parch", "Sex"])

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

<ipython-input-75-845cf99fce63>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sex"] = df["Sex"].apply(lambda x: int(x == "female"))
<ipython-input-75-845cf99fce63>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sex"] = df["Sex"].apply(lambda x: int(x == "female"))
<ipython-input-75-845cf99fce63>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

### Task 1.1

Create dataloaders for each train, val, and test datasets. Select `batch_size`

## Task 2
Write a class for a network that can classify if the person surviced Titanic catastrophy. For now use only one linear layer and try out both [BCE](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html) and [BCEWithLogitsLoss](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html). Connect WandB logger to track the loss. 

Train and evalute the model.

In [ ]:
class Network(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.l1 = torch.nn.Linear(4, 1, dtype=torch.float64)

  def configure_optimizers(self):
    return torch.optim.SGD(self.parameters(), lr=0.02, momentum=0.9)

  def training_step(self, batch, batch_idx):
    x, y = batch
    logits = self(x).view(-1)
    loss = torch.nn.functional.binary_cross_entropy_with_logits(logits, y)
    self.log("train_loss", loss)
    return loss

  def validation_step(self, batch, batch_idx):
    x, y = batch
    logits = self(x).view(-1)
    loss = torch.nn.functional.binary_cross_entropy_with_logits(logits, y)

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    x = self.l1(x)
    return x 

In [ ]:
model = Network()

In [ ]:
EPOCHS = 50

In [ ]:
trainer = pl.Trainer(max_epochs=EPOCHS)
trainer.fit(model, train_dataloader, val_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type   | Params
--------------------------------
0 | l1   | Linear | 5     
--------------------------------
5         Trainable params
0         Non-trainable params
5         Total params
0.000     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


In [ ]:
# Evaluate the model

In [ ]:
# getting the predictions

y_pred = []
y_true = []

for x_batch, y_batch in test_dataloader:
    logits = model.forward(x_batch)
    probs = torch.nn.functional.sigmoid(logits)
    y_pred.extend((probs > 0.5).int().view(-1).tolist())
    y_true.extend(y_batch.tolist())

/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1967: UserWarning: nn.func

In [ ]:
# checking the accuracy
print(classification_report(y_true, y_pred))
print(accuracy_score(y_true, y_pred))

              precision    recall  f1-score   support

         0.0       0.79      0.86      0.83       212
         1.0       0.76      0.67      0.72       144

    accuracy                           0.78       356
   macro avg       0.78      0.77      0.77       356
weighted avg       0.78      0.78      0.78       356



NameError: ignored